# Part 1: Read Data


In [ ]:
import pandas as pd
import numpy as np

pd.set_option('chained_assignment', None)
ds = pd.read_csv("S:...//inspector _list.csv")
ds = ds.rename(columns={'ELEVATOR Employee': 'Name'})

ds["Name"] = ds["Name"].str.replace('[^\x00-\x7F]','')
ds["Name"] = ds["Name"].map(str.upper)
ds["Name"] = ds["Name"].map(str.strip)


#ACCELLA data
df = pd.read_csv("..\\Inspector Spatial Analysis Oct 16 - 22.csv")

df = df[["INSPECTOR_NAME_FML#", "DATE_INSPECTION", "MILEAGE_START", "MILEAGE_END", "DATE_SCHEDULED", "STATUS", "INSPECTOR_DEPT", \
         "HOUSE#", "STREET_NAME", "LONGITUDE", "LATITUDE", "CITY"]]

df = df.rename(columns={'INSPECTOR_NAME_FML#': 'NAME'})

df["ADDRESS"] = df["HOUSE#"] + " " + df["STREET_NAME"] + " " + df["CITY"]

df["ADDRESS"] = df["ADDRESS"].astype(str)
df["ADDRESS"] = df["ADDRESS"].map(str.upper)
df["NAME"] = df["NAME"].astype(str)
df["NAME"] = df["NAME"].map(str.upper)
df["NAME"] = df["NAME"].map(str.strip)
df["NAME"] = df["NAME"].str.replace('[^\x00-\x7F]','')
df = df[df["NAME"] != "NAN"]
df = df.reset_index(drop=True)

df['DATE_SCHEDULED'] = pd.to_datetime(df['DATE_SCHEDULED'])

print len(df)

#get week date time of interest:

#start_time = str(lv["Start Time"][i].month) + '-' + str(lv["Start Time"][i].day) + '-' + str(lv["Start Time"][i].year)
#end_time = str(lv["End Time"][i].month) + '-' + str(lv["End Time"][i].day) + '-' + str(lv["End Time"][i].year)

start_time = '10-17-2016'
end_time = '10-21-2016'

mask = (df['DATE_SCHEDULED'] >= start_time) & (df['DATE_SCHEDULED'] <= end_time)

#mask = (df['DATE_SCHEDULED'] >= start_time)
df = df.loc[mask]
df = df.reset_index(drop=True)


#print len(df)

#mask = df['DATE_SCHEDULED'] <= end_time
#df = df.loc[mask]
print len(df)

In [ ]:
#Field Force Management 


dfm = pd.read_csv("..\\Worker_Stops_Time_Oct2016_EDIT.csv")

dfm = dfm[["Group", "Name","Start Time", "Departure Time", "Event Type","Event Name", "Address", "Stop Duration", "GPS Distance", \
             "Travel Duration", "Lat", "Lon"]]


#print "length of concate", len(dfm)

dfm['Name'] = dfm['Name'].map(lambda x: x.strip('0123456789'))
dfm["Name"] = dfm["Name"].map(str.strip)
dfm["Name"] = dfm["Name"].str.replace('[^\x00-\x7F]','')

dfm['Start Time'] = pd.to_datetime(dfm['Start Time'])


start_time = '10-17-2016'
end_time = '10-21-2016'

mask = (dfm['Start Time'] >= start_time) & (dfm['Start Time'] <= end_time)

dfm = dfm.loc[mask]

dfm = dfm.reset_index(drop=True)




In [ ]:
dfm

# Part 1b: Create List of Inspectors from Accela to Analyze

In [ ]:
#get the list of supervisor names and create new dataframe from accela output with those names
names = []
for i in range(0, len(ds)):
    names.append(ds["Name"][i])

names = set(names)
names = list(names)
names = sorted(names)
print "name list"
print names

df2 = pd.DataFrame(columns=("NAME","DATE_INSPECTION", "MILEAGE_START", "MILEAGE_END", "DATE_SCHEDULED", "STATUS", \
                            "INSPECTOR_DEPT", "HOUSE#", "STREET_NAME", "LONGITUDE", "LATITUDE", "CITY","ADDRESS"))

#get the names in accela
for i in range(0, len(df)):
    if df["NAME"][i] in names:
        #print df["NAME"][i]
        df2.loc[i] = [df["NAME"][i],df["DATE_INSPECTION"][i], df["MILEAGE_START"][i], df["MILEAGE_START"][i], \
                     df["DATE_SCHEDULED"][i],df["STATUS"][i], df["INSPECTOR_DEPT"][i], df["HOUSE#"][i], \
                     df["STREET_NAME"][i],df["LONGITUDE"][i], df["LATITUDE"][i], df["CITY"][i], df["ADDRESS"][i]  ]   


df2 = df2.reset_index(drop=True)




# Part 1c: Create List of Inspectors in FFM dataset to Analyze

In [ ]:
ffm_accela_names = []

for i in range(0, len(dfm)):
    ffm_accela_names.append(dfm["Name"][i])
        
ffm_accela_names = set(ffm_accela_names)
ffm_accela_names = list(ffm_accela_names)
ffm_accela_names = sorted(ffm_accela_names)
#print ffm_accela_names
#print len(ffm_accela_names)

dfm2 = pd.DataFrame(columns=("Name","Start Time", "Departure Time", "Event Type", "Event Name", "Address", \
                            "Stop Duration", "GPS Distance", "Travel Duration", "Lat", "Lon"))

#get the names in accela
for i in range(0, len(dfm)):
    if dfm["Name"][i] in names:
        #print df["NAME"][i]
        dfm2.loc[i] = [dfm["Name"][i],dfm["Start Time"][i], dfm["Departure Time"][i], dfm["Event Type"][i], \
                     dfm["Event Name"][i],dfm["Address"][i], dfm["Stop Duration"][i], dfm["GPS Distance"][i], \
                     dfm["Travel Duration"][i],dfm["Lat"][i], dfm["Lon"][i]]   


dfm2 = dfm2.reset_index(drop=True)




In [ ]:
#dfm2[0:100]

In [ ]:
print names
print len(names)

# Part 2: Clean Addresses

In [ ]:
pd.set_option('chained_assignment', None)

#get rid of zip code in field force dataframe
dfm2['Address'] = dfm2['Address'].astype(str)
#dfm['Address'] = dfm['Address'].map(lambda x: x.strip('0123456789'))
dfm2["Address"] = dfm2["Address"].map(str.strip)
dfm2["Address"] = dfm2["Address"].map(str.upper)

for i in range(0, len(dfm2)):
    text = dfm2["Address"][i]
    #print "original text:", text
    #remove last element in string
    text = text.rsplit(' ', 1)[0]
    #print "split text:", text
    #print
    dfm2["Address"][i] = text

    
#further cleaning of field force
dfm2["Address"] = dfm2["Address"].str.replace(',', '')
dfm2["Address"] = dfm2["Address"].str.replace('NEW YORK NY', 'MANHATTAN')
dfm2["Address"] = dfm2["Address"].str.replace('LONG ISLAND CITY', 'QUEENS')
dfm2["Address"] = dfm2["Address"].str.replace('SUNNYSIDE', 'QUEENS')
dfm2["Address"] = dfm2["Address"].str.replace('JAMAICA', 'QUEENS')
dfm2["Address"] = dfm2["Address"].str.replace(' NY', '')
#dfm2["Address"] = dfm2["Address"].str.replace('TH ', ' ')

#clean accela data
df2["ADDRESS"] = df2["ADDRESS"].map(str.strip)
df2["ADDRESS"] = df2["ADDRESS"].map(str.upper)

# Part 3: Address Matching

In [ ]:

from operator import itemgetter
from difflib import SequenceMatcher as SM
#s1 = '213'
#s2 = '250'
#print SM(None, s1, s2).ratio()

df2["Visit"] = "NA"
df2["Closest Address Visited"] = "NA"

df2["Fuzzy Score"] = 0.0

for i in range(0, len(df2)):
#for i in range(0, 10):
    #df3 =df2[df2['NAME'] == names[i]]
    #df3 = df3.reset_index(drop=True)
    
    #get name from accella data and subset ffm data
    name = df2["NAME"][i]
    dfm3 = dfm2[dfm2["Name"] == name]
    dfm3 = dfm3.reset_index(drop=True)
    
    

    #print df3
    #print
    #print dfm2
    
    ratio_list = []
    
    for j in range(0, len(dfm3)):
        if dfm3.empty:
            print "dfm3 empty"
            
        
        s1 = str(df2["ADDRESS"][i])
        s2 = str(dfm3["Address"][j])
        ratio =  SM(None, s1, s2).ratio()
        ratio = round(ratio, 2)
        
        if dfm3["Address"][j] != 'NAN':
            ratio_list.append([dfm3["Address"][j], ratio])
        
        if ratio >= 0.89:
            df2["Visit"][i] = "YES"
        
        elif (ratio < 0.89 and df2["Visit"][i] == "NA"):
            df2["Visit"][i] = "NO"
       
        
    #ratio_list =  sorted(ratio_list, reverse=True)
    
    ratio_list = sorted(ratio_list, key=itemgetter(1), reverse=True)
    
    
    if df2["Visit"][i] != "NA":
        #print 
        print len(ratio_list)
        
        if len(ratio_list) != 0:
            print ratio_list[0]
            df2["Closest Address Visited"][i] = ratio_list[0][0]
            df2["Fuzzy Score"][i] = ratio_list[0][1]
    
   

# Part 4: Location Using Nearest Neighbor Lat/Lon Coordinates

In [ ]:
from operator import itemgetter

from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    #convert into feet before returning
    return km*3280.84



#print haversine(lon1, lat1, lon2, lat2)

df2["Visit Spatial"] = "NA"
df2["Closest Address Visited Spatial"] = "NA"

df2["Closest Distance (ft)"] = 0.0

for i in range(0, len(df2)):
#for i in range(2, 4):

    #get name from accella data and subset ffm data
    name = df2["NAME"][i]
    dfm3 = dfm2[dfm2["Name"] == name]
    dfm3 = dfm3.reset_index(drop=True)
    
    #drop NANs
    #dfm3 = dfm3.dropna()
    dfm3 = dfm3.reset_index(drop=True)

    #print dfm3
    
    lon1 = df2["LONGITUDE"][i]
    lat1 = df2["LATITUDE"][i]
    #print df3
    #print
    #print dfm2
    
    feet_list = []
    
    for j in range(0, len(dfm3)):
        if dfm3.empty:
            print "dfm3 empty"
     
        lon2 = dfm3["Lon"][j]
        lat2 = dfm3["Lat"][j]

        dist = haversine(lon1, lat1, lon2, lat2)
        dist = round(dist)
        
        #only create list if have address, because cannot sort dist NANs properly
        if dfm3["Address"][j] != 'NAN':
            feet_list.append([dfm3["Address"][j], dist])

        if dist <= 1000:
            df2["Visit Spatial"][i] = "YES"
        
        elif (dist > 1000 and df2["Visit Spatial"][i] == "NA"):
            df2["Visit Spatial"][i] = "NO"
       
    #sort to get the minimum distance 
    feet_list =  sorted(feet_list, key=itemgetter(1), reverse=False)
    #print feet_list
    #print
    
    #ratio_list = sorted(ratio_list, key=itemgetter(1), reverse=True)
    
    
    if df2["Visit Spatial"][i] != "NA":
        print feet_list[0]
        df2["Closest Address Visited Spatial"][i] = feet_list[0][0]
        df2["Closest Distance (ft)"][i] = feet_list[0][1]
    
    """
    if df2["Visit"][i] != "NA":
        print 
        print ratio_list[0]
        df2["Closest Address Visited"][i] = ratio_list[0][0]
        df2["Fuzzy Score"][i] = ratio_list[0][1]
    """

In [ ]:
df2 = df2[df2["Closest Address Visited"] != 'NA']

df2 = df2.reset_index(drop=True)

df2


In [ ]:
df2.to_csv("S..\Analysis 1.csv")